In [1]:
import numpy as np
from numpy.linalg import (svd,det,norm)
import sys
from os.path import join,dirname,abspath
from Structures import Quaternion

In [2]:
def wahbas_problem(ref_los, body_los):
	"""
		Returns R rotation matrix from body to reference frame
	"""
	A = ref_los.T
	E = body_los.T
	B = A@E.T
	U,S,V = svd(B)
	M = np.eye(3)
	M[2,2] = det(U)*det(V.T)
	return U@M@V

In [3]:
# Camera position
cam_pos = np.array([3384526.963722019,-1445726.0382287714,1051739.9176159338]) # meters

# Target Position
target_pos = np.array([0,0,0]) # meters

# Align Z axis with target
z_axis = target_pos-cam_pos
z_axis = z_axis/np.linalg.norm(z_axis)
#print(z_axis)

# Use y aligned with Moon center to find x axis
#y_axis = cam_pos/np.linalg.norm(cam_pos)

# Use y aligned with global Z to find x axis
y_axis = np.array([0,0,-1])

# Calculate x axis
x_axis = np.cross(y_axis,z_axis)
#x_axis = np.array([0,1,0])

# Recalculate y axis
y_axis = np.cross(z_axis,x_axis)

#print("X = {}".format(x_axis))
#print("Y = {}".format(y_axis))
#print("Z = {}".format(z_axis))

local = np.eye(3)
globl = np.array([x_axis,y_axis,z_axis])

R = wahbas_problem(local,globl)
print("DCM:\n{}".format(R.T))
print()
quat = Quaternion()
quat.fromDCM(R.T)
print("Quaternion:\n{}".format(quat))
print()
print("Position:\n[{},{},{}]".format(cam_pos[0],cam_pos[1],cam_pos[2]))

DCM:
[[ 3.92820504e-01  2.52683190e-01 -8.84219010e-01]
 [ 9.19615165e-01 -1.07935517e-01  3.77700772e-01]
 [ 5.31618360e-17 -9.61509818e-01 -2.74770577e-01]]

Quaternion:
[0.50252, 0.66624, 0.43989, -0.33179]

Position:
[3384526.963722019,-1445726.0382287714,1051739.9176159338]


In [4]:
# Get pointing vector from DCM
boresight = np.array([0,0,1])
pvec = quat.toDCM()@boresight
print("Pointing Vector is: {}".format(pvec))

Pointing Vector is: [-0.88421901  0.37770077 -0.27477058]


In [6]:
# Alternative Quaternion
alt_q_arr = np.array([0.5603790202671888,-0.39212942619470076,0.71493184004968,0.14519755796768524])
alt_quat = Quaternion()
alt_quat.fromArray(alt_q_arr)
alt_pvec = alt_quat.toDCM()@boresight
print("Alternate Pointing Vector is: {}".format(alt_pvec))

Alternate Pointing Vector is: [-0.91513808 -0.23186949 -0.32978605]
